# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

Выберете источник данных, который вам симпатичен. По умолчанию, предлагаем использовать данные о погоде в Москве: 
https://www.gismeteo.ru/weather-moscow-4368/. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

In [1]:
import pandas as pd
import re
import urllib.request

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from tqdm import tqdm

In [2]:
url = "https://www.gismeteo.ru/weather-moscow-4368/now/"
req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
html = urllib.request.urlopen(url)
html = html.read()

In [3]:
temperText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[1]/div/div/div[1]/div[3]/div[1]/span').extract()[0]

In [4]:
regInt = re.compile('[\-\+0-9]{2,}')
regFloat = re.compile('[\,0-9]{2,}')

In [5]:
temperature =  regInt.search(temperText)[0] + regFloat.search(temperText)[0]
temperature = float(temperature.replace(',', '.'))
temperature

1.8

In [6]:
windSpeedText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[2]/div/div[6]/div[1]/div').extract()[0]

In [7]:
regRusText = re.compile('[А-Яа-я]{1,}')
regNum = re.compile('[0-9]{1,}')

In [8]:
windSpeedText = regRusText.search(windSpeedText)[0] + " " + regNum.search(windSpeedText)[0] + " м/с." 

In [9]:
windSpeedText

'Ветер 2 м/с.'

In [10]:
regRusText.search(windSpeedText)


<_sre.SRE_Match object; span=(0, 5), match='Ветер'>

In [11]:
wetText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[2]/div/div[6]/div[3]').extract()[0]

In [12]:
wetText = regRusText.search(wetText)[0] + " " + regNum.search(wetText)[0] + " %."

In [13]:
wetText

'Влажность 92 %.'

## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [15]:
def weather():
    import pymorphy2
    import pandas as pd
    import re
    import urllib.request
    from time import gmtime, strftime
    from scrapy.selector import Selector
    from scrapy.http import HtmlResponse
    from tqdm import tqdm
    
    morph = pymorphy2.MorphAnalyzer()
    ruDegree = morph.parse('градус')[0];
    
    url = "https://www.gismeteo.ru/weather-moscow-4368/now/"
    req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
    html = urllib.request.urlopen(url)
    html = html.read()
    
    regInt = re.compile('[\-\+0-9]{2,}')
    regFloat = re.compile('[\,0-9]{2,}')
    regRusText = re.compile('[А-Яа-я]{1,}')
    regNum = re.compile('[0-9]{1,}')
    
    temperText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[1]/div/div/div[1]/div[3]/div[1]/span').extract()[0]
    temperature =  regInt.search(temperText)[0] + regFloat.search(temperText)[0]
    temperature = float(temperature.replace(',', '.'))
    
    
    
    windSpeedText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[2]/div/div[6]/div[1]/div').extract()[0]
    windSpeedText = regRusText.search(windSpeedText)[0] + " " + regNum.search(windSpeedText)[0] + " м/с." 
    
    wetText = Selector(text=html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[2]/div/div[6]/div[3]').extract()[0]
    wetText = regRusText.search(wetText)[0] + " " + regNum.search(wetText)[0] + " %."
    
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    print('Сейчас в Москве {0} {1} (\xB0C).'.format(temperature, ruDegree.make_agree_with_number(temperature).word))
    print(windSpeedText)
    print(wetText)

In [16]:
weather()

2017-12-30 18:24:39
Сейчас в Москве 1.8 градусов (°C).
Ветер 2 м/с.
Влажность 92 %.


## Часть 3 (необязательная)
Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели (например, "сегодня похолодало на Х градусов по сравнению со вчера").